In [1]:
from layers.autoencoder.vae import AudioAutoencoder,OobleckDecoder,OobleckEncoder
import os
import torch



In [2]:


model_state = torch.load("../models/Nov29.pth",map_location=torch.device('cpu'))
model = AudioAutoencoder(sample_rate=16000,downsampling_ratio=2048)
model.state_dict(model_state)
total_params = sum(p.numel() for p in model.parameters())
print(f"\nTotal Parameters: {total_params:,}")



Total Parameters: 159,088,798


/var/folders/tl/kzvy4pzs20d7mtb0c82b429h0000gn/T/ipykernel_6750/395464002.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_state = torch.load("../models/Nov29.pth",

In [15]:
#torch.ones(1, 16384)
input = torch.cat((torch.ones((1, 16384//2))*100,-1000*torch.ones(1, 16384//2)),dim=-1)
latents = model.encode(input)
output = model.decode(latents)


print(input.shape)
print(latents.shape)
print(output.shape)

from layers.tools.losses import MultiResolutionSTFTLoss
f = MultiResolutionSTFTLoss()

error = f(input.unsqueeze(1), output.unsqueeze(1))
print(error)
print(output)
#tensor([[-0.0015, -0.0020, -0.0029,  ..., -0.0011,  0.0007,  0.0027]],
#tensor([[-0.0015, -0.0020, -0.0029,  ..., -0.0011,  0.0007,  0.0027]],
#tensor([[-0.0077, -0.0068, -0.0131,  ...,  0.0104,  0.0168,  0.0140]],

'''
    #TODO:
        1. decoder-only training
        2. turn long sequence into stacked batch to put it in encoder
        3. EMA consideration on DiT
'''

torch.Size([1, 16384])
torch.Size([1, 128, 4])
torch.Size([1, 16384])
tensor(1947.5219, grad_fn=<DivBackward0>)
tensor([[ 0.1154,  0.2712,  0.3870,  ..., -0.0283,  0.0606,  0.0165]],
       grad_fn=<SqueezeBackward1>)


'\n    #TODO:\n        1. decoder-only training\n        2. turn long sequence into stacked batch to put it in encoder\n        3. EMA consideration on DiT\n'

In [4]:
from pprint import pprint

for mo in model.encoder.parameters():
    mo.requires_grad=False